# Data Profiling of QVI_data csv

In [2]:
import pandas as pd  

df = pd.read_csv('../../raw_data/QVI_data.csv')
df.head(5)

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 264834 entries, 0 to 264833
Data columns (total 12 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   LYLTY_CARD_NBR    264834 non-null  int64  
 1   DATE              264834 non-null  object 
 2   STORE_NBR         264834 non-null  int64  
 3   TXN_ID            264834 non-null  int64  
 4   PROD_NBR          264834 non-null  int64  
 5   PROD_NAME         264834 non-null  object 
 6   PROD_QTY          264834 non-null  int64  
 7   TOT_SALES         264834 non-null  float64
 8   PACK_SIZE         264834 non-null  int64  
 9   BRAND             264834 non-null  object 
 10  LIFESTAGE         264834 non-null  object 
 11  PREMIUM_CUSTOMER  264834 non-null  object 
dtypes: float64(1), int64(6), object(5)
memory usage: 24.2+ MB


In [4]:
print(df.describe(include='all'))

        LYLTY_CARD_NBR        DATE      STORE_NBR        TXN_ID  \
count     2.648340e+05      264834  264834.000000  2.648340e+05   
unique             NaN         364            NaN           NaN   
top                NaN  2018-12-24            NaN           NaN   
freq               NaN         939            NaN           NaN   
mean      1.355488e+05         NaN     135.079423  1.351576e+05   
std       8.057990e+04         NaN      76.784063  7.813292e+04   
min       1.000000e+03         NaN       1.000000  1.000000e+00   
25%       7.002100e+04         NaN      70.000000  6.760050e+04   
50%       1.303570e+05         NaN     130.000000  1.351365e+05   
75%       2.030940e+05         NaN     203.000000  2.026998e+05   
max       2.373711e+06         NaN     272.000000  2.415841e+06   

             PROD_NBR                               PROD_NAME       PROD_QTY  \
count   264834.000000                                  264834  264834.000000   
unique            NaN              

## Additional Data Quality Inspections

1. Duplicate records

In [5]:
df.duplicated().sum()

np.int64(1)

In [6]:
df = df.drop_duplicates()
df.duplicated().sum()

np.int64(0)

2. Existance of 272 stores check

In [7]:
df['STORE_NBR'].nunique()

272

## Corrections

In [8]:
df['DATE'] = pd.to_datetime(df['DATE'])

# Exploratory Data Analysis

The client has selected store numbers 77, 86 and 88 as trial stores and wants control stores to be established stores that are operational for the entire observation period.

We would want to match trial stores to control stores that are similar to the trial store prior to the trial period of Feb 2019 in terms of :
- Monthly overall sales revenue
- Monthly number of customers
- Monthly number of transactions per customer

In [9]:
# extracting year-month 

df['YEAR_MONTH'] = df['DATE'].dt.to_period('M')

df.head(5)

,LYLTY_CARD_NBR,DATE,STORE_NBR,TXN_ID,PROD_NBR,PROD_NAME,PROD_QTY,TOT_SALES,PACK_SIZE,BRAND,LIFESTAGE,PREMIUM_CUSTOMER,YEAR_MONTH
0,1000,2018-10-17,1,1,5,Natural Chip Compny SeaSalt175g,2,6.0,175,NATURAL,YOUNG SINGLES/COUPLES,Premium,2018-10
1,1002,2018-09-16,1,2,58,Red Rock Deli Chikn&Garlic Aioli 150g,1,2.7,150,RRD,YOUNG SINGLES/COUPLES,Mainstream,2018-09
2,1003,2019-03-07,1,3,52,Grain Waves Sour Cream&Chives 210G,1,3.6,210,GRNWVES,YOUNG FAMILIES,Budget,2019-03
3,1003,2019-03-08,1,4,106,Natural ChipCo Hony Soy Chckn175g,1,3.0,175,NATURAL,YOUNG FAMILIES,Budget,2019-03
4,1004,2018-11-02,1,5,96,WW Original Stacked Chips 160g,1,1.9,160,WOOLWORTHS,OLDER SINGLES/COUPLES,Mainstream,2018-11


In [10]:
summary = (
    df.groupby(['STORE_NBR', 'YEAR_MONTH']).agg(
        SALES_REVENUE = ('TOT_SALES', 'sum'),
        CUSTOMER_COUNT = ('LYLTY_CARD_NBR', 'nunique'),
        TOTAL_TXN = ('TXN_ID', 'nunique')
    ).reset_index()
)

summary['TXN_PER_CUSTOMER'] = summary['TOTAL_TXN'] / summary['CUSTOMER_COUNT']

summary.head(10)

,STORE_NBR,YEAR_MONTH,SALES_REVENUE,CUSTOMER_COUNT,TOTAL_TXN,TXN_PER_CUSTOMER
0,1,2018-07,206.9,49,52,1.061224
1,1,2018-08,176.1,42,43,1.023810
2,1,2018-09,278.8,59,62,1.050847
3,1,2018-10,188.1,44,45,1.022727
4,1,2018-11,192.6,46,47,1.021739
5,1,2018-12,189.6,42,47,1.119048
6,1,2019-01,154.8,35,36,1.028571
7,1,2019-02,225.4,52,55,1.057692
8,1,2019-03,192.9,45,49,1.088889
9,1,2019-04,192.9,42,43,1.023810


In [18]:
pre_trial_data = summary[summary['YEAR_MONTH'] < '2019-02']

store_avg_data = pre_trial_data.groupby('STORE_NBR').agg(
    AVG_MONTHLY_SALES = ('SALES_REVENUE', 'mean'),
    AVG_CUSTOMER_FLOW = ('CUSTOMER_COUNT', 'mean'),
    AVG_TXN_PER_CUSTOMER = ('TXN_PER_CUSTOMER', 'mean')
).reset_index()

In [19]:
trial_stores = [77, 86, 88]

trial_store_data = store_avg_data[store_avg_data['STORE_NBR'].isin(trial_stores)]

control_store_candidates = store_avg_data[~store_avg_data['STORE_NBR'].isin(trial_stores)]

In [20]:
# months of trial

print(f"Date range of data is from {df['DATE'].min()} to {df['DATE'].max()}")

Date range of data is from 2018-07-01 00:00:00 to 2019-06-30 00:00:00


> The trial ran from February 2019 to June 2019. A total of 5 full months.

In [24]:
trial_store_data.head()

,STORE_NBR,AVG_MONTHLY_SALES,AVG_CUSTOMER_FLOW,AVG_TXN_PER_CUSTOMER
76,77,242.714286,42.714286,1.057898
85,86,874.264286,99.571429,1.256935
87,88,1340.514286,125.714286,1.217688


In [23]:
control_store_candidates.head(5)

,STORE_NBR,AVG_MONTHLY_SALES,AVG_CUSTOMER_FLOW,AVG_TXN_PER_CUSTOMER
0,1,198.128571,45.285714,1.046852
1,2,161.214286,38.857143,1.051386
2,3,1075.164286,106.285714,1.172833
3,4,1303.857143,121.285714,1.219322
4,5,819.957143,93.000000,1.255987


## Correlation Calculation

In [26]:
pre_trial_data.head()

,STORE_NBR,YEAR_MONTH,SALES_REVENUE,CUSTOMER_COUNT,TOTAL_TXN,TXN_PER_CUSTOMER
0,1,2018-07,206.9,49,52,1.061224
1,1,2018-08,176.1,42,43,1.023810
2,1,2018-09,278.8,59,62,1.050847
3,1,2018-10,188.1,44,45,1.022727
4,1,2018-11,192.6,46,47,1.021739


In [28]:
time_series_sales = pre_trial_data.pivot(
    index= 'STORE_NBR',
    columns= 'YEAR_MONTH',
    values= 'SALES_REVENUE'
)

time_series_customers = pre_trial_data.pivot(
    index='STORE_NBR',
    columns='YEAR_MONTH',
    values='CUSTOMER_COUNT'
)

time_series_txn = pre_trial_data.pivot(
    index='STORE_NBR',
    columns='YEAR_MONTH',
    values='TXN_PER_CUSTOMER'
)


In [31]:
def corr_calculation():

    results = {}

    for store in trial_stores:
        corr_sales = time_series_sales.T.corr()[store]
        corr_customers = time_series_customers.T.corr()[store]
        corr_txn = time_series_txn.T.corr()[store]

        avg_corr = (corr_sales + corr_customers + corr_txn) / 3

        avg_corr = avg_corr.drop(store) #removing correlation done with itself
        results[store] = avg_corr.sort_values(ascending=False).head(3) #top 3 match

    return results

## Magnitude Distance Calculation

In [36]:
from sklearn.preprocessing import StandardScaler

def mag_distance():

    results = {}

    metrics_df = store_avg_data[['STORE_NBR', 'AVG_MONTHLY_SALES', 'AVG_CUSTOMER_FLOW', 'AVG_TXN_PER_CUSTOMER']].set_index('STORE_NBR')

    scaler = StandardScaler()

    metrics_scaled = pd.DataFrame(
        scaler.fit_transform(metrics_df),
        index= metrics_df.index,
        columns= metrics_df.columns
    )

    for store in trial_stores:
        distances = ((metrics_scaled - metrics_scaled.loc[store]).abs()).drop(store)

        results[store] = distances.nsmallest(3)

    return results

## Finding Control Stores